In [47]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hw-texts/baseline_submit.csv
/kaggle/input/hw-texts/train.csv
/kaggle/input/hw-texts/test.csv


In [48]:
! pip install pymorphy2[fast] rnnmorph pymystem3 nltk razdel

In [49]:
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, cross_val_score, cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, roc_auc_score, classification_report, precision_score, recall_score, make_scorer
import joblib
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import razdel
from pymorphy2 import MorphAnalyzer
from sklearn.linear_model import SGDClassifier

In [50]:
train = pd.read_csv('/kaggle/input/hw-texts/train.csv', sep=';')
test = pd.read_csv('/kaggle/input/hw-texts/test.csv', sep=';')

In [51]:
train.fillna(0)
train.replace([np.inf, -np.inf], 0, inplace=True)
test.fillna(0)
test.replace([np.inf, -np.inf], 0, inplace=True)

In [52]:
X_train, X_test, y_train, y_test = train_test_split(train['text'], train['label'], test_size=0.3, random_state=42)

In [53]:
%matplotlib inline

import tqdm
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

def evaluate_vectorizer(model, vectorizer, X_train, X_test, y_train, y_test = None):
    train_vectors = vectorizer.fit_transform(X_train)
    test_vectors = vectorizer.transform(X_test)
    
    clf = model(random_state=42)
    
    clf.fit(train_vectors, y_train)
    
    predictions = clf.predict(test_vectors)
    if (y_test is not None):
        print(clf, classification_report(y_test, predictions), sep = '\n')
    return predictions

In [54]:
clfs = [LinearSVC, DecisionTreeClassifier, RandomForestClassifier, AdaBoostClassifier, SGDClassifier] 


In [55]:
for i in range(len(clfs)):
    clf = clfs[i]
    test_predictions = evaluate_vectorizer(clf, TfidfVectorizer(min_df=2), X_train, X_test, y_train, y_test)

LinearSVC(random_state=42)
              precision    recall  f1-score   support

           0       0.75      0.64      0.69      4500
           1       0.98      0.97      0.98     10486
           2       0.81      0.88      0.84      8632

    accuracy                           0.87     23618
   macro avg       0.85      0.83      0.84     23618
weighted avg       0.87      0.87      0.87     23618

DecisionTreeClassifier(random_state=42)
              precision    recall  f1-score   support

           0       0.51      0.43      0.47      4500
           1       0.94      0.93      0.94     10486
           2       0.69      0.76      0.72      8632

    accuracy                           0.77     23618
   macro avg       0.71      0.71      0.71     23618
weighted avg       0.77      0.77      0.77     23618

RandomForestClassifier(random_state=42)
              precision    recall  f1-score   support

           0       0.79      0.20      0.32      4500
           1       0.9

In [56]:
def tokenize_with_razdel(text):
    tokens = [token.text for token in razdel.tokenize(text)]
    
    return tokens

In [57]:
evaluate_vectorizer(LinearSVC, TfidfVectorizer(min_df=2, tokenizer=tokenize_with_razdel), X_train, X_test, y_train, y_test);

LinearSVC(random_state=42)
              precision    recall  f1-score   support

           0       0.76      0.65      0.70      4500
           1       0.99      0.98      0.98     10486
           2       0.82      0.88      0.85      8632

    accuracy                           0.88     23618
   macro avg       0.85      0.84      0.84     23618
weighted avg       0.88      0.88      0.88     23618



In [58]:
pymorphy = MorphAnalyzer()
def lemmatize_with_pymorphy(tokens):
    lemms = [pymorphy.parse(token)[0].normal_form for token in tokens]
    return lemms
tfidf_vectorizer = TfidfVectorizer(
    min_df=2, 
    tokenizer=lambda text: lemmatize_with_pymorphy(tokenize_with_razdel(text)),
)

In [59]:
test_predictions = evaluate_vectorizer(LinearSVC, CountVectorizer(min_df=2), X_train, X_test, y_train, y_test)

LinearSVC(random_state=42)
              precision    recall  f1-score   support

           0       0.69      0.64      0.67      4500
           1       0.98      0.96      0.97     10486
           2       0.80      0.84      0.82      8632

    accuracy                           0.86     23618
   macro avg       0.82      0.82      0.82     23618
weighted avg       0.86      0.86      0.86     23618



/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [60]:
test_predictions = evaluate_vectorizer(LinearSVC, TfidfVectorizer(min_df=2, tokenizer=tokenize_with_razdel), X_train, X_test, y_train, y_test)

LinearSVC(random_state=42)
              precision    recall  f1-score   support

           0       0.76      0.65      0.70      4500
           1       0.99      0.98      0.98     10486
           2       0.82      0.88      0.85      8632

    accuracy                           0.88     23618
   macro avg       0.85      0.84      0.84     23618
weighted avg       0.88      0.88      0.88     23618



In [61]:
test_predictions = evaluate_vectorizer(LinearSVC, tfidf_vectorizer, X_train, X_test, y_train, y_test)

LinearSVC(random_state=42)
              precision    recall  f1-score   support

           0       0.77      0.67      0.72      4500
           1       0.99      0.98      0.98     10486
           2       0.83      0.89      0.86      8632

    accuracy                           0.89     23618
   macro avg       0.86      0.85      0.85     23618
weighted avg       0.89      0.89      0.89     23618



In [62]:
predictions = evaluate_vectorizer(LinearSVC, tfidf_vectorizer, train['text'], test['text'], train['label'])

test['label'] = predictions
test[['id', 'label']].to_csv('Pigalov.csv', index=None)